<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #9: Modelado de temas
`Fecha de entrega: Mayo 6, 2021. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.

In [3]:
# Cargar datos
import pandas as pd

data = pd.read_csv("reviews_vidjew_es.csv")
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


In [4]:
import re 

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

[12 pts] Punto 1: Hacer pre-procesamiento del texto

In [7]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = texto.split() # Tokenizar
    texto = [palabra for palabra in texto if palabra not in stopwords_sp]
    return texto

data['Pre-Processed'] = data['review_body'].apply(lambda texto: pre_procesado(texto))

data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,Pre-Processed
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


[10 pts] Punto 2: Modelo de LDA

In [9]:
! pip install pyLDAvis
! pip install pyLDAvis --upgrade

Requirement already up-to-date: pyLDAvis in c:\users\57314\anaconda3\lib\site-packages (3.3.1)


In [11]:
import pyLDAvis.gensim_models
from gensim.models import LdaModel 
from gensim.corpora import Dictionary
from pprint import pprint

In [24]:
# Crear una represantación de los documentos en forma de diccionario
dictionary = Dictionary(data['Pre-Processed'].values)

# Filtrar palabras muy frecuentes o infrecuentes
dictionary.filter_extremes(no_below=5, no_above=0.5) 

# Corpus
corpus = [dictionary.doc2bow(text) for text in data['Pre-Processed'].values]

# Entrenar el modelo
model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = 5, passes=500)

In [25]:
model.print_topics(num_words=9)

[(0,
  '0.046*"calidad" + 0.036*"precio" + 0.033*"buena" + 0.024*"foto" + 0.022*"juego" + 0.020*"regalo" + 0.018*"pulsera" + 0.016*"bastante" + 0.015*"bonita"'),
 (1,
  '0.041*"si" + 0.028*"producto" + 0.027*"mando" + 0.024*"puede" + 0.021*"mas" + 0.020*"mejor" + 0.017*"bien" + 0.017*"ps" + 0.015*"calidad"'),
 (2,
  '0.034*"amazon" + 0.030*"perfecto" + 0.022*"comprar" + 0.022*"día" + 0.020*"después" + 0.019*"producto" + 0.018*"envío" + 0.018*"rápido" + 0.017*"tiempo"'),
 (3,
  '0.044*"bien" + 0.035*"llegado" + 0.027*"juego" + 0.024*"buen" + 0.019*"llegó" + 0.019*"producto" + 0.018*"día" + 0.017*"funciona" + 0.014*"precio"'),
 (4,
  '0.041*"bien" + 0.027*"bonito" + 0.023*"calidad" + 0.016*"dos" + 0.016*"si" + 0.015*"tamaño" + 0.015*"cadena" + 0.014*"plata" + 0.014*"colgante"')]

[25 pts] Punto 3: Visualización de LDA

In [26]:
lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

[3 pts] Preguntas
¿Qué conclusiones puede sacar de los resultados de los últimos tres talleres, usando diferentes modelos pero los mismos datos?

La diferencia entre los tres últimos taller es que en el primero existía etiqueta en la variable dependiente, es decir, se tenían definidas dos clases, en los dos últimos talleres con k-means y LDA se definió que lo mejor era separar en 5 clases. En los dos últimos talleres se evidencia una diferencia entre las palabras más importantes en cada grupo.